# **Extract DayMET Data**

By Bridget Bittmann

Date created: March 28, 2022

Date editted: May 4, 2022

Purpose: This script extracts DayMET data from Google Earth Engine using the Earth Engine API in Python. This script also copied over a SSEBop ET data dataset and calculated zonal stats based on polygons. 

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=49sSsfdneMDYgCPWEIsErDhsyqyVeTl02Ci3QSKY3Pk&tc=1ojUY4FGn7ZAAiBTawxTkj7HwzVSb4AKm-kPJ6Se2X8&cc=wzmfTCBPVUfIGac4fv7W73lYnMT5IGPCVVrz36rn6Pc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJpsHhd9MkutrwydN-U79xr8h1oCbtt6i5uWz9pUmKuUPuS1YZdeaUc

Successfully saved authorization token.


In [2]:
!pip install geopandas
import geopandas as gpd #import independent shapefile
import json #for metadata of shapefile
import os #for file paths
import numpy as np #for stats and arrays
import pandas as pd #for dataframes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.8 MB/s 
     |████████████████████████████████| 6.3 MB 38.3 MB/s 
     |████████████████████████████████| 16.7 MB 52.9 MB/s 


In [3]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Go to correct folder in Google Drive
%cd drive/MyDrive/spatial_colab/datasets/
%ls

/content/drive/MyDrive/spatial_colab/datasets
climate_stats/         irrigation_companies/  lcmap_files/   subset_test_shp/
diversion_timeseries/  irrig_lbrb/            masked/
extra_figures/         IrrMapper/             output_files/
hydromet_data/         LBRB_shp/              POUs/


In [5]:
## ------------------------------------------------------- ## 
## 1. Import shapefile and start/end dates to clip dataset ##
## ------------------------------------------------------- ## 

start_end = pd.read_csv('climate_stats/start_end_dates.csv')
shp_file = 'POUs/POUs_EDIT_060622_Merge.shp'
subset = emap.shp_to_ee(shp_file) # converts shapefile to feature in GEE
bpbc = emap.shp_to_ee('subset_test_shp/BPBC_Divisions/BPBC_Divisions-IDTM.shp')

map=emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(bpbc, 0, 2), {}, 'POU')
map.addLayerControl()
map

In [15]:
## ------------------------------------------------- ## 
## IMPORT PRECIP, TEMP, and ET FOR IRRIGATION SEASON ## 
## ------------------------------------------------- ## 

years = np.arange(1987,2021)

# Empty lists to store images
ir_tmp = []
ir_pr = []
mean_max = []
et_irrig = []
mar_tmp = []
mar_prcp = []
mar_et = []

# Import image collection, subset to shapefile, take a statistics for a period of time, 
# and append the image to the designated list
for i in range(len(start_end)):
  mar = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-03-01'), (str(years[i])+'-03-31'))
  m_tmp  = mar.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index': (str(years[i])+'-03')})
  m_prcp  = mar.select('prcp').map(lambda image: image.clip(subset)).mean().set({'system:index': (str(years[i])+'-03')})
  et_mar = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(years[i])+'-03-01'), (str(years[i])+'-3-31'))
  et_m = et_mar.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(years[i])}) 
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate(start_end['StartDate'][i], start_end['EndDate'][i]) #get image collection for irrigation season
  daymet_hot = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-06-01'), (str(years[i])+'-8-31')) #get image collection for June-Aug
  et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(years[i])+'-04-01'), str(years[i])+'-10-31')
  et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(years[i])}) # sum et and convert to meters 
  mxtmp = daymet_hot.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index':str(years[i])}) #select temp to analyze hot months and take mean
  tmp = daymet.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index':str(start_end['StartDate'][i])}) #select max temp to analyze and take mean
  pr = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index': str(start_end['StartDate'][i])}) #select precip to analyze and sum
  ir_tmp.append(tmp)
  ir_pr.append(pr)
  mean_max.append(mxtmp) 
  et_irrig.append(et)
  mar_tmp.append(m_tmp)
  mar_prcp.append(m_prcp)
  mar_et.append(et_m)

# Convert lists of images to image collection for zonal stats command
et_irrig = ee.ImageCollection(et_irrig)
ir_tmp = ee.ImageCollection(ir_tmp)
ir_pr = ee.ImageCollection(ir_pr)
means_max_temp = ee.ImageCollection(mean_max)

In [16]:
# Convert lists of images to image collection for zonal stats command
et_irrig = ee.ImageCollection(et_irrig)
ir_tmp = ee.ImageCollection(ir_tmp)
ir_pr = ee.ImageCollection(ir_pr)
means_max_temp = ee.ImageCollection(mean_max)
et_march = ee.ImageCollection(mar_et)
tmp_march = ee.ImageCollection(mar_tmp)
prcp_march = ee.ImageCollection(mar_prcp)

In [ ]:
## ------------------------------------------------- ## 
## IMPORT PRECIP, TEMP, and ET FOR IRRIGATION SEASON ## 
## ------------------------------------------------- ## 

## BPBC ##

years = np.arange(1987,2021)

# Empty lists to store images
ir_tmp = []
ir_pr = []
mean_max = []
et_irrig = []

# Import image collection, subset to shapefile, take a statistics for a period of time, 
# and append the image to the designated list
for i in range(len(start_end)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate(start_end['StartDate'][i], start_end['EndDate'][i]) #get image collection for irrigation season
  daymet_hot = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-06-01'), (str(years[i])+'-8-31')) #get image collection for June-Aug
  et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(years[i])+'-04-01'), str(years[i])+'-10-31')
  et = et_data.map(lambda image: image.clip(bpbc)).sum().multiply(0.00001).set({'system:index': str(years[i])}) # sum et and convert to meters 
  mxtmp = daymet_hot.select('tmax').map(lambda image: image.clip(bpbc)).mean().set({'system:index':str(years[i])}) #select temp to analyze hot months and take mean
  tmp = daymet.select('tmax').map(lambda image: image.clip(bpbc)).mean().set({'system:index':str(start_end['StartDate'][i])}) #select max temp to analyze and take mean
  pr = daymet.select('prcp').map(lambda image: image.clip(bpbc)).sum().set({'system:index': str(start_end['StartDate'][i])}) #select precip to analyze and sum
  ir_tmp.append(tmp)
  ir_pr.append(pr)
  mean_max.append(mxtmp) 
  et_irrig.append(et)

# Convert lists of images to image collection for zonal stats command
et_irrig = ee.ImageCollection(et_irrig)
ir_tmp = ee.ImageCollection(ir_tmp)
ir_pr = ee.ImageCollection(ir_pr)
means_max_temp = ee.ImageCollection(mean_max)

In [ ]:
## ---------------------------------------------------------------------- ##
## 3. IMPORT THE DAYMET DATA FOR PRECIPITATION PRIOR TO IRRIGATION SEASON ##
## ---------------------------------------------------------------------- ##

# This will provide insight into antecedent moisture conditions for a POU.

years = np.arange(1987,2021)
ant_pr = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i]-1)+'-10-31'), start_end['StartDate'][i]) #get image collection
  prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index':str(i+1)}) #select the bands to analyze
  ant_pr.append(prcp) #calculate the mean across all pixels

ant_precip = ee.ImageCollection(ant_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(ant_precip, precip_vis, 'prcp')
Map

In [ ]:
## ---------------------------------------------------------------------- ##
## 3. IMPORT THE DAYMET DATA FOR PRECIPITATION PRIOR TO IRRIGATION SEASON ##
## ---------------------------------------------------------------------- ##

## BPBC ##
# This will provide insight into antecedent moisture conditions for a POU.

years = np.arange(1987,2021)
ant_pr = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i]-1)+'-10-31'), start_end['StartDate'][i]) #get image collection
  prcp = daymet.select('prcp').map(lambda image: image.clip(bpbc)).sum().set({'system:index':str(i+1)}) #select the bands to analyze
  ant_pr.append(prcp) #calculate the mean across all pixels

ant_precip = ee.ImageCollection(ant_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(ant_precip, precip_vis, 'prcp')
Map

In [17]:
## ------------------------ ##
## 4. CALCULATE ZONAL STATS ##
## ------------------------ ##

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

out_stats = os.path.join('climate_stats/JAtemp_stats.csv')
emap.zonal_statistics(means_max_temp, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/ir_tmp_stats.csv')
emap.zonal_statistics(ir_tmp, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/irrig_precip_stats.csv')
emap.zonal_statistics(ir_pr, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/ant_precip_stats.csv')
emap.zonal_statistics(ant_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/et.csv')
emap.zonal_statistics(et_irrig, subset, out_stats, statistics_type='MEAN', scale=30)

out_stats = os.path.join('climate_stats/et_march.csv')
emap.zonal_statistics(et_march, subset, out_stats, statistics_type='MEAN', scale=30)

out_stats = os.path.join('climate_stats/tmp_march.csv')
emap.zonal_statistics(tmp_march, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/prcp_march.csv')
emap.zonal_statistics(prcp_march, subset, out_stats, statistics_type='MEAN', scale=30)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/et_march.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/tmp_march.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/prcp_march.csv


In [ ]:
## ------------------------ ##
## 4. CALCULATE ZONAL STATS ##
## ------------------------ ##

# BPBC # 

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

out_stats = os.path.join('climate_stats/JAtemp_stats_bpbc.csv')
emap.zonal_statistics(means_max_temp, bpbc, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/ir_tmp_stats_bpbc.csv')
emap.zonal_statistics(ir_tmp, bpbc, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/irrig_precip_stats_bpbc.csv')
emap.zonal_statistics(ir_pr, bpbc, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/ant_precip_stats_bpbc.csv')
emap.zonal_statistics(ant_precip, bpbc, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/et_bpbc.csv')
emap.zonal_statistics(et_irrig, bpbc, out_stats, statistics_type='MEAN', scale=30)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/JAtemp_stats_bpbc.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/ir_tmp_stats_bpbc.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/irrig_precip_stats_bpbc.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/ant_precip_stats_bpbc.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/et_bpbc.csv


In [ ]:
## ---------------------------------------------- ##
## 5. CREATE CLIMATE STAT FOR EACH POU AND EXPORT ##
## ---------------------------------------------- ##

years = np.arange(1987,2021)
ir_precip = pd.read_csv('climate_stats/irrig_precip_stats.csv')
ant_precip = pd.read_csv('climate_stats/ant_precip_stats.csv')
JA_temp = pd.read_csv('climate_stats/JAtemp_stats.csv')
irrig_temp = pd.read_csv('climate_stats/ir_tmp_stats.csv')
et_irrig = pd.read_csv('climate_stats/et.csv')
## MARCH INFO## 


names = et_irrig['WaterRight']

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['ant_prcp'] = ant_precip.iloc[i,0:34].values
  df['irrig_prcp'] = ir_precip.iloc[i,0:34].values 
  df['irrig_temp'] = irrig_temp.iloc[i,0:34].values
  df['JuneAug_temp'] = JA_temp.iloc[i,0:34].values
  df['et'] = et_irrig.iloc[i,0:34].values
  out_path = os.path.join('climate_stats/final/'+names[i]+'_climate.csv')
  df.to_csv(out_path)


In [ ]:
## ---------------------------------------------- ##
## 5. CREATE CLIMATE STAT FOR EACH POU AND EXPORT ##
## ---------------------------------------------- ##

## BPBC ##

years = np.arange(1987,2021)
ir_precip = pd.read_csv('climate_stats/irrig_precip_stats_bpbc.csv')
ant_precip = pd.read_csv('climate_stats/ant_precip_stats_bpbc.csv')
JA_temp = pd.read_csv('climate_stats/JAtemp_stats_bpbc.csv')
irrig_temp = pd.read_csv('climate_stats/ir_tmp_stats_bpbc.csv')
et_irrig = pd.read_csv('climate_stats/et_bpbc.csv')

names = et_irrig['DivisionNa']

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['ant_prcp'] = ant_precip.iloc[i,0:34].values
  df['irrig_prcp'] = ir_precip.iloc[i,0:34].values 
  df['irrig_temp'] = irrig_temp.iloc[i,0:34].values
  df['JuneAug_temp'] = JA_temp.iloc[i,0:34].values
  df['et'] = et_irrig.iloc[i,0:34].values
  out_path = os.path.join('climate_stats/bpbc_final/'+names[i]+'_climate.csv')
  df.to_csv(out_path)
